# Hurt labeling

## Import Library

In [1]:
import os
import pickle

import pandas as pd
import re

from konlpy.tag import Komoran, Okt

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model

from tqdm import tqdm

## Hyper_parameters

In [2]:
# Colab
# DATA_DIR = '/content/drive/MyDrive/Colab Notebooks/miso'

# Local
DATA_DIR = '/Users/inseoklee/Desktop/miso/data'
SAVE_DIR = '/Users/inseoklee/Desktop/miso/data'
LOAD_DIR = '/Users/inseoklee/Desktop/miso/model/CNN'

# MAX_LEN -> 패딩 단계에서 설정

## Load Data

In [3]:
df_hurt = pd.read_csv(DATA_DIR + '/label_hurt.csv')

In [4]:
df_hurt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14258 entries, 0 to 14257
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    14258 non-null  object
 1   label   14258 non-null  object
dtypes: object(2)
memory usage: 222.9+ KB


## Tokenize

In [5]:
def preprocessing(text, okt, remove_stopwords=False, stop_words=[]):
    # 함수의 인자는 다음과 같다.
    # text: 전처리할 텍스트
    # okt: okt객체를 반복적으로 생성하지 않고 미리 생성한 후 인자로 받는다.
    # remove_stopwords: 불용어를 제거할지 여부 선택. 기본값은 False
    # stop_words: 불용어 사전은 사용자가 직접 입력.

    # 1. 한글 및 공백을 제외한 문자를 모두 제거
    text = re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]", "", text)

    # 2. okt 객체를 활용하여 형태소 단위로 나눈다
    word_text = okt.morphs(text, stem=True)

    if remove_stopwords:
        word_text = [token for token in word_text if (not token in stop_words)] #and len(token) > 1]

    return word_text

In [6]:
stop_words = set(['은', '는', '이', '가', '하', '아', '것', '들', '의', '있', '되', '수', '보', '주', '등', '한'])
okt = Okt()

[0.003s][warning][os,thread] Attempt to protect stack guard pages failed (0x000000016b268000-0x000000016b274000).
[0.003s][warning][os,thread] Attempt to deallocate stack guard pages failed.


## Load Model and Tokenizer

In [7]:
def load_tokenizer(path):
    with open(path, 'rb') as f:
        tokenizer = pickle.load(f)
    return tokenizer

model_name = 'trained_model.h5'
tokenizer_name = 'tokenizer.pickle'
model_path = os.path.join(LOAD_DIR, model_name)
tokenizer_path = os.path.join(LOAD_DIR, tokenizer_name)

model = load_model(model_path)
tokenizer = load_tokenizer(tokenizer_path)

Metal device set to: Apple M1


2022-07-13 16:40:40.665081: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-07-13 16:40:40.665171: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


## Labeling

In [8]:
def getResult(predict):
    highest = 0
    highestIdx = 0
    for x in range (0,len(predict)) :
        if(predict[x] > highest) :
            highestIdx = x
            highest =predict[x]
    return highestIdx

angry
sad
fear
disgusting
neutral
happy
surprised

In [9]:
def predict_sentiment(text, model):
    tokens = []
    if type(text) == str:
        tokens.append(preprocessing(text, okt, remove_stopwords=True, stop_words=stop_words))
    else:
        tokens.append([]) #string이 아니면 비어있는 값 추가
    token_sequences = tokenizer.texts_to_sequences(tokens)
    if(len(token_sequences) == 0) :
        return "너무 짧아"
    padded_sequences = pad_sequences(token_sequences, maxlen=15)
    predict = model.predict(padded_sequences, verbose=0)
    # print(predict)
    result = getResult(predict[0])
    if result == 0 :
        return 'angry'
    elif result == 1 :
        return 'sad'
    elif result == 2 :
        return 'fear'
    elif result == 3 :
        return 'disgusting'
    elif result == 4 :
        return 'neutral'
    elif result == 5 :
        return 'happy'
    elif result == 6 :
        return 'surprised'

In [10]:
label_hurt = []

for idx in tqdm(range(len(df_hurt['text'])), desc="Labeling", mininterval=0.01):
    label = predict_sentiment(df_hurt['text'][idx], model)
    label_hurt.append(label)

Labeling:   0%|          | 0/14258 [00:00<?, ?it/s]2022-07-13 16:40:42.884406: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-07-13 16:40:42.925234: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
Labeling: 100%|██████████| 14258/14258 [06:06<00:00, 38.86it/s]


In [11]:
df_hurt['label'] = label_hurt
df_hurt[:20]

,text,label
0,새로 간 치과에서 조금 썩기만 한 이를 임플란트를 하라고 했어. 과잉진료는 안 한다더니.,disgusting
1,부인이 건강검진 결과가 나왔는데 건강 상태가 좋질 않아 충격을 받은 모양이야.,neutral
2,나 병원에 가 봐야 할 것 같아.,sad
3,아들 녀석이 학원을 제대로 다니는가 싶었는데 그게 아니었어. 정말 속상해.,sad
4,치과의 과잉 진료로 안 해도 되는 임플란트까지 하게 되어 큰돈을 지불했어.,sad
5,산책하던 중에 갑자기 자전거가 튀어나와 팔목이 부러졌지 뭐야.,sad
6,시댁에 가면 의지할 사람이 아무도 없어.,sad
7,병을 앓으면서 친하게 지내던 동호회 사람들과 멀어졌어.,angry
8,딸이 또 공무원 시험에 떨어졌대. 애쓰는 걸 지켜봐서 그런지 내 마음도 괴롭네.,sad
9,진짜 공부 못하던 친구가 우리 회사에 합격했대. 너무 배 아파.,sad


### Re labeling

In [12]:
df_hurt.drop(df_hurt[df_hurt['label'] == 'happy'].index, inplace=True)

In [13]:
df_hurt['label'].unique()

array(['disgusting', 'neutral', 'sad', 'angry', 'fear', 'surprised'],
      dtype=object)

## Save data

In [14]:
df_hurt.to_csv(SAVE_DIR+'/relabel_hurt.csv', encoding='utf-8-sig', index=False)